In [1]:
import qiskit as qc
from qiskit import QuantumCircuit
import numpy as np

/home/jupyter-arturolopez/.local/lib/python3.9/site-packages/qiskit/providers/aer/pulse/qutip_extra_lite/fastsparse.py:57: DeprecationWarning: Please use `upcast` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_index_dtype)
/home/jupyter-arturolopez/.local/lib/python3.9/site-packages/qiskit/providers/aer/pulse/qutip_extra_lite/fastsparse.py:57: DeprecationWarning: Please use `isdense` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_index_dtype)
/home/jupyter-arturolopez/.local/lib/python3.9/site-packages/qiskit/providers/aer/pulse/qutip_extra_lite/fastsparse.py:57: DeprecationWarning: Please use `isscalarlike` from the `scipy.sparse` namespace, the `scipy.sparse.sputils` namespace is deprecated.
  from scipy.sparse.sputils import (upcast, isdense, isscalarlike, get_inde

In [2]:
bare = qc.QuantumCircuit(3,1) 
#this is the sample circuit that we perform RC for in this code
bare.h(0)
bare.i(1)
bare.cx(0,1) 
bare.i(2) 
#end of the first cycle; each inputted circuit must be coded like this
bare.barrier()

bare.y(0)
bare.i(1)
bare.h(2)
bare.cx(1,2)
#end of second cycle; the cx command can be placed anywhere within the barriers,
# but the single qubit gates must be assigned to qubits in ascending order
bare.barrier()

bare.i(0)
bare.x(1)
bare.i(2)
#the last cycle must contain no hard (cx) gates, only easy (single-qubit) gates.
bare.draw()

#another restriction is the use of only H,X,Z,Y,I, T, and CNOT gates initially

┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ Y ├──────░─┤ I ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ I ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ I ├──────░─┤ H ├┤ X ├─░─┤ I ├
     └───┘      ░ └───┘└───┘ ░ └───┘
c: 1/═══════════════════════════════

In [3]:
def parsing(QuantumCircuit):
    qasm = QuantumCircuit.qasm()
    lis = []
    lin = ""
    for s in qasm:
        if s == ";":
            lis.append(lin)
            lin=""
        elif s == "\n":
            pass
        else:
            lin += s
    
    lis.pop(0)
    lis.pop(0)
    #getting rid of the first two lines of the qasm code
    #could also make a while loop to pop() the lines that don't start with qreg
    
    if "qreg" in lis[0]:
        qreg=lis[0]
        numQubits=int(qreg[-2])
        lis.pop(0)
    else:
        print("There is an error in getting the number of qubits")
    
    if "creg" in lis[0]: #a restriction we do not have is the number of 
        lis.pop(0) # classical bits in the inputted bare circuit
    
    easyGates = []
    numCycles = 1
    for i in range(len(lis)):
        line = lis[i]
        if "h" in line:
            easyGates.append("H")
        elif "x" in line and "c" not in line:
            easyGates.append("X")
        elif "y" in line:
            easyGates.append("Y")
        elif "z" in line:
            easyGates.append("Z")
        elif "t" in line:
            easyGates.append("T")
        elif "id" in line:
            easyGates.append("I")
        elif "barrier" in line:
            numCycles += 1
        elif "cx" in line:
            continue
        else:
            print(f"unexpected line or gate in qasm interpretation in line {i} ")
    
    controls = []
    targets = []
    
    for i in range(len(lis)):
        line = lis[i]
        if "cx" in line:
            for j in range(len(line)):
                if line[j] == "]":
                    a=line[j-1]
                    if j == 6:
                        controls.append(a)
                    elif j==11:
                        targets.append(a)
                    else:
                        print("something weird happening with cx")
    return numQubits, numCycles, easyGates, controls, targets

In [4]:
(numQubits, numCycles, easyGates, controls, targets)=parsing(bare)

In [5]:
def twirling_gates(numQubits, numCycles): #assigns T_k gates from Twirling set
    twirling_set = ["X","Y","Z","I"]
    random_twirling = []
    for i in range(numQubits*(numCycles-1)):
        random_twirling.append(np.random.choice(twirling_set))
    for i in range(numQubits):
        random_twirling.append("I")
    return random_twirling

In [6]:
random_twirling = twirling_gates(numQubits, numCycles)
random_twirling

['Y', 'X', 'X', 'X', 'Y', 'I', 'I', 'I', 'I']

In [7]:
def inverting_operators(random_twirling, targets, numQubits):
    target_indeces = []
    for i in range(len(targets)): #getting indeces of inverting_matrices that need to change
        target_indeces.append(str(int(targets[i])+ numQubits*i))
    
    inverting_gates = []
    for i in range(numQubits):
        inverting_gates.append("I")
    for i in range(len(random_twirling)-numQubits):
        if str(i) in target_indeces:
            #do multiplication formula:: Honestly, nevermind: just match with if state
            if random_twirling[i]=="X":
                inverting_gates.append("I")
            elif random_twirling[i]=="Y":
                inverting_gates.append("Z") #i think its a z rotation of maybe 180 degrees. Matrix is [i,0],[0,-i]
            elif random_twirling[i]=="Z":   #the fact that it's a z rotation makes sense w pen method.
                inverting_gates.append("Y") #not sure if correct. matrix is [0,-1],[1,0]
            elif random_twirling[i]=="I":
                inverting_gates.append("X")
            else:
                print("unrecognized gate before cx target qubit in prev. cycle")
        else:
            inverting_gates.append(random_twirling[i])
    return target_indeces, inverting_gates
        

In [8]:
(target_indeces, inverting_gates)=inverting_operators(random_twirling, targets, numQubits)
inverting_gates

['I', 'I', 'I', 'Y', 'I', 'X', 'X', 'Y', 'X']

In [9]:
def gate2mat(gate):
    if gate == "X":
        gate_matrix = np.array([[0,1],
                                [1,0]])
    elif gate == "Y":
        gate_matrix = np.array([[0,-1j],
                                [1j,0]])
    elif gate == "Z":
        gate_matrix = np.array([[1,0],
                                [0,-1]])
    elif gate == "H":
        gate_matrix = 1/np.sqrt(2) *np.array([[1,1],
                                [1,-1]])
    elif gate == "I":
        gate_matrix = np.array([[1,0],
                                [0,1]])
    elif gate == "T":
        gate_matrix = np.array([[1,0],
                                [0,np.exp(1j*np.pi/4)]])
    else:
        print(f"matrix {gate} not recognized")
        gate_matrix = np.array([[0,0],[0,0]])
    return gate_matrix

In [10]:
def reduction(easyGates, random_twirling, inverting_gates, numQubits, numCycles):
    compiled_gates = np.zeros((numQubits*numCycles,2,2),dtype = 'complex_')
    
    for i in range(numQubits*numCycles):
        mat1=gate2mat(inverting_gates[i])
        mat2=gate2mat(easyGates[i])
        mat3=gate2mat(random_twirling[i])
        a=np.dot(mat2,mat1)
        compiled_gates[i]=np.dot(mat3,a)
    
    #want to return an array, matrix of 2x2x(however 
    #many easy gates we have in the circuit= numQubits*numCycles)
    return compiled_gates

In [11]:
(compiled_gates)=reduction(easyGates, random_twirling, inverting_gates, numQubits, numCycles)
compiled_gates

array([[[ 0.        -0.70710678j,  0.        +0.70710678j],
        [ 0.        +0.70710678j, -0.        +0.70710678j]],

       [[ 0.        +0.j        ,  1.        +0.j        ],
        [ 1.        +0.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  1.        +0.j        ],
        [ 1.        +0.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  1.        +0.j        ],
        [ 1.        +0.j        ,  0.        +0.j        ]],

       [[ 0.        +0.j        ,  0.        -1.j        ],
        [ 0.        +1.j        ,  0.        +0.j        ]],

       [[ 0.70710678+0.j        ,  0.70710678+0.j        ],
        [-0.70710678+0.j        ,  0.70710678+0.j        ]],

       [[ 0.        +0.j        ,  1.        +0.j        ],
        [ 1.        +0.j        ,  0.        +0.j        ]],

       [[ 0.        +1.j        ,  0.        +0.j        ],
        [ 0.        +0.j        ,  0.        -1.j        ]],

       [[ 0.        +0.j

In [26]:
#turn matrix into gate: will need to know how to generalize the forms of the
# possible matrices
def mat2gate(a): #given a 2x2 np.array()
    gate=[]
    gateList = ["X", "Y", "Z", "H", "I", "T", "H","H","H","H","H","H","H","H","X","X","X","Z","Z","Z",
                "Yn90", "I", "Yn90","Yn90"]
    
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,-1j],[1j,0]])
    Z = np.array([[1,0],[0,-1]])
    H = 1/np.sqrt(2) *np.array([[1,1],[1,-1]])
    I = np.array([[1,0],[0,1]])
    T = np.array([[1,0],[0,np.exp(1j*np.pi/4)]])
    
    #extra gates I've been adding after running code:
    H2 = 1/np.sqrt(2)*np.array([[-1,-1],[-1,1]])
    H3 = 1/np.sqrt(2)*np.array([[-1j,-1j],[-1j,1j]])
    H4 = 1/np.sqrt(2) *np.array([[1j,1j],[1j,-1j]])
    H5 = 1/np.sqrt(2)*np.array([[1,-1],[1,1]])
    H6 = 1/np.sqrt(2) *np.array([[-1,1],[-1,-1]])
    H7 = 1/np.sqrt(2)*np.array([[1j,-1j],[1j,1j]])
    H8 = 1/np.sqrt(2) *np.array([[-1j,1j],[-1j,-1j]])
    H9 = 1/np.sqrt(2) *np.array([[1,-1],[-1,-1]])
    
    X2 = np.array([[0,-1j],[-1j,0]])
    X3 = np.array([[0,-1],[-1,0]])
    X4 = np.array([[0,1j],[1j,0]])
    
    Z2 = np.array([[1j,0],[0,-1j]])
    Z3 = np.array([[-1,0],[0,1]])
    Z4 = np.array([[-1j,0],[0,1j]])
    
    Yn90 = 1/np.sqrt(2) *np.array([[1,1],[-1,1]])
    
    I2 = np.array([[1j,0],[0,1j]])
    
    minusOpp = 1/np.sqrt(2) *np.array([[-1,1],[1,1]])
    minusOpp2 = 1/np.sqrt(2) *np.array([[-1j,1j],[1j,1j]]) #this gate is awkward because
    #we can get to this location with a simple Yn90 gate, but to get the orientation
    # of the vector precisely right, we would need at least two gates, ex. "H" then a "Y".
    #For now, Ill just associate this matrix with the Yn90 rotation.
    
    matList = [X, Y, Z, H, I, T, H2,H3,H4,H5,H6,H7,H8,H9,X2,X3,X4,Z2, Z3, Z4,
               Yn90, I2, minusOpp, minusOpp2] #only works for these specific definitions
    for i,item in enumerate(matList):
        if (a == item).all():
            gate = gateList[i]
    if gate == []:
        print(f"the following gate needs to be defined:{a}")
    return gate

In [27]:
def instructions(compiled_gates):
    randEZgates = []
    for i in range(len(compiled_gates)):
        randEZgates.append(mat2gate(compiled_gates[i]))
        
    return randEZgates

In [28]:
randEZgates = instructions(compiled_gates)
print(randEZgates)
print(random_twirling)
print(inverting_gates)
bare.draw()

['Yn90', 'X', 'X', 'X', 'Y', 'Yn90', 'X', 'Z', 'X']
['Y', 'X', 'X', 'X', 'Y', 'I', 'I', 'I', 'I']
['I', 'I', 'I', 'Y', 'I', 'X', 'X', 'Y', 'X']


┌───┐      ░ ┌───┐      ░ ┌───┐
q_0: ┤ H ├──■───░─┤ Y ├──────░─┤ I ├
     ├───┤┌─┴─┐ ░ ├───┤      ░ ├───┤
q_1: ┤ I ├┤ X ├─░─┤ I ├──■───░─┤ X ├
     ├───┤└───┘ ░ ├───┤┌─┴─┐ ░ ├───┤
q_2: ┤ I ├──────░─┤ H ├┤ X ├─░─┤ I ├
     └───┘      ░ └───┘└───┘ ░ └───┘
c: 1/═══════════════════════════════

In [29]:
mat1=gate2mat("H") #for testing the previous function 
mat2=gate2mat("Y") #and determining which arrays to account for
plop=np.dot(mat2,mat1)
mat3=gate2mat("Y")
np.dot(mat3,plop)
plop

array([[ 0.-0.70710678j,  0.+0.70710678j],
       [ 0.+0.70710678j, -0.+0.70710678j]])

In [30]:
def final_product(randEZgates, controls, targets, numQubits):
    RCqc=QuantumCircuit(numQubits)
    for j in range(len(targets)):
        for i in range(numQubits):
            var = randEZgates[i+j*numQubits]
            if var == "X":
                RCqc.x(i)
            elif var == "Y":
                RCqc.y(i)
            elif var == "Z":
                RCqc.z(i)
            elif var == "H":
                RCqc.h(i)
            elif var == "I":
                RCqc.id(i)
            elif var == "T":
                RCqc.t(i)
            elif var == "Yn90": #the weird class of compiled gates:
                RCqc.ry(-np.pi/2,i)
            else:
                print(f"randEXgates has unexpected gate from mat2gate function at index {i}")
            #all the other gates we could possibly get, including paramaterized gates somehow
        RCqc.cx(int(controls[j]),int(targets[j]))
        RCqc.barrier()
    
    for i in range(len(targets)*numQubits,len(randEZgates)):
        var = randEZgates[i]
        if var == "X":
            RCqc.x(i-len(targets)*numQubits)
        elif var == "Y":
            RCqc.y(i-len(targets)*numQubits)
        elif var == "Z":
            RCqc.z(i-len(targets)*numQubits)
        elif var == "H":
            RCqc.h(i-len(targets)*numQubits)
        elif var == "I":
            RCqc.id(i-len(targets)*numQubits)
        elif var == "T":
            RCqc.t(i-len(targets)*numQubits)
        else:
            print(f"randEXgates has unexpected gate from mat2gate function at index {i-len(targets)*numQubits-1}")
    #one final loop for last round of randEZgates (not attached to hard gate cycle)
    return RCqc

In [31]:
qc = final_product(randEZgates, controls, targets, numQubits)
qc.draw()

┌──────────┐      ░    ┌───┐          ░ ┌───┐
q_0: ┤ RY(-π/2) ├──■───░────┤ X ├──────────░─┤ X ├
     └──┬───┬───┘┌─┴─┐ ░    ├───┤          ░ ├───┤
q_1: ───┤ X ├────┤ X ├─░────┤ Y ├──────■───░─┤ Z ├
        ├───┤    └───┘ ░ ┌──┴───┴───┐┌─┴─┐ ░ ├───┤
q_2: ───┤ X ├──────────░─┤ RY(-π/2) ├┤ X ├─░─┤ X ├
        └───┘          ░ └──────────┘└───┘ ░ └───┘

In [18]:
#run bare circuit and qc (RC) 

In [19]:
hey = QuantumCircuit(1) #for testing how to apply a parematrized gate
hey.ry(np.pi/2, 0)
hey.draw()

┌─────────┐
q_0: ┤ RY(π/2) ├
     └─────────┘

In [20]:
from qiskit import visualization #also for testing, but it didnt work
from qiskit.visualization import visualize_transition 

In [21]:
#lingering questions: ask Phil if orientation of
#the vector matters, or just its position in the Bloch Sphere